In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr
from sklearn.feature_selection import SelectKBest, f_regression
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import shap
import pandas as pd
from src.llm import Generator
from IPython.display import Markdown
from tqdm.notebook import tqdm
import os


## Interpretability

In this notebook, we demonstrate how the interpretation plots were generated using SHAP (SHapley Additive exPlanations). Due to the computational and memory demands of SHAP's `KernelExplainer`, we attempted to execute the process on a high-performance cluster equipped with 64 CPU cores and 384 GB of RAM. Despite these substantial resources, the computational requirements remained prohibitive, necessitating an alternative approach.

To address this limitation, we performed the SHAP analysis on smaller subsets of features. Specifically, we utilized the best-performing pipeline with a feature selection threshold of \( k = 500 \). This pipeline achieved a Kaggle score of **0.49**, that is still a winning score in the competition, while also enabling us to integrate the interpretability component effectively.

In [3]:
X = pd.read_csv('./data/train.csv', index_col=0)
X_submission = pd.read_csv('./data/test.csv', index_col=0)
y = pd.read_csv('./data/train_targets.csv', index_col=0)
y = y["AAC"]
X_submission_index = X_submission.index
features = X.columns
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
pipeline =  Pipeline([
        ('scaler', StandardScaler()),
        ('feature_selector', SelectKBest(score_func=f_regression, k=500)),
        ('regressor', SVR(kernel='linear', C=1.0, epsilon=0.30, tol=1e-3))
    ])

In [5]:
pipeline.fit(X_train, y_train.values.ravel())

Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selector',
                 SelectKBest(k=500,
                             score_func=<function f_regression at 0x00000219DF923100>)),
                ('regressor', SVR(epsilon=0.3, kernel='linear'))])

In [6]:
y_train_pred = pipeline.predict(X_train)
y_test_pred = pipeline.predict(X_test)
y_submission = pipeline.predict(X_submission)

# Evaluate the model
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

# Compute Spearman correlation
spearman_corr, _ = spearmanr(y_test, y_test_pred)

print(f"Train MSE: {train_mse:.5f}")
print(f"Test MSE: {test_mse:.5f}")
print(f"Spearman correlation: {spearman_corr:.5f}")

Train MSE: 0.03689
Test MSE: 0.04277
Spearman correlation: 0.41856


In [7]:
y_submission = pipeline.predict(X_submission)
submission = pd.DataFrame({"sampleId": X_submission_index.str.replace("CL", "TS"), "AAC": y_submission})
submission.to_csv('./submissions/submission_shap_svr.csv', index=False)

In [8]:
# Extract the transformed data
scaler = pipeline.named_steps['scaler']
feature_selector = pipeline.named_steps['feature_selector']
regressor = pipeline.named_steps['regressor']

# Prepare submission set and background data (apply same transformations as pipeline)
X_scaled = scaler.transform(X_submission) 
X_selected = feature_selector.transform(X_scaled)
selected_feature_names = X_train.columns[feature_selector.get_support()]

X_train_scaled = scaler.transform(X_train)
X_train_selected = feature_selector.transform(X_train_scaled)
background_data = shap.kmeans(X_train_selected, k=50)

c:\Users\jerem\anaconda3\envs\ml\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


In [9]:
explainer = shap.KernelExplainer(regressor.predict, background_data)

In [10]:
sorted_indices = np.argsort(y_submission)
top_indices = sorted_indices[-5:]
bottom_indices = sorted_indices[:5:]
median_index = sorted_indices[len(sorted_indices) // 2]
median_indices = [median_index - 2, median_index - 1, median_index, median_index + 1, median_index + 2]
list_to_explain = np.concatenate((X_selected[top_indices], X_selected[median_indices], X_selected[bottom_indices]))

In [11]:
shap_values = explainer(list_to_explain)
shap_values.feature_names = selected_feature_names

  0%|          | 0/15 [00:00<?, ?it/s]

## Create the shap waterfall plots and beeswarm

This allows to create the plots showing the most important features for top 5, median 5 and bottom 5 predictions. 

In [12]:
for i in range(len(list_to_explain)):
    fig = shap.plots.waterfall(shap_values[i], max_display=5, show=False)
    
    name = f"top_{i}" if i < 5 else f"bottom_{i}" if i >= 10 else f"median_{i}"
    plt.savefig(f'./shap_plots/{name}.png', bbox_inches="tight") 
    plt.clf()

<Figure size 800x400 with 0 Axes>

In [13]:
fig = shap.plots.beeswarm(shap_values, show=False)
plt.savefig('./shap_plots/beeswarm.png', bbox_inches="tight") 

In [14]:
shap_tables = []

for i in range(len(list_to_explain)):
    shap_val = shap_values[i]
    
    feature_importance = pd.DataFrame({
        'Feature': shap_val.feature_names,
        'SHAP Value': shap_val.values,
        'Feature Value': shap_val.data
    })
    
    top_features = feature_importance.reindex(feature_importance['SHAP Value'].abs().sort_values(ascending=False).index)[:4]
    shap_tables.append(top_features)


## LLM interpretation

Now a small llama model with 8B parameters is loaded locally and used for inference in order to provide more detailed explanations about the top features and what they represent.


In [15]:
model_key = "meta-llama/Meta-Llama-3.1-8B-Instruct"
model_family = "llama"

generator = Generator(local_compute=True, model_key=model_key, model_family=model_family)
_, tokenizer = generator.load_model()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model meta-llama/Meta-Llama-3.1-8B-Instruct loaded on device cuda:0


In [27]:
system_prompt = """You are a highly knowledgeable medical expert with expertise in genomics, bioinformatics, and machine learning. 
You are tasked with explaining the impact of specific genetic features on a prediction model to a patient or researcher. 
Your explanation should be highly formal, precise, and delivered in a professional tone. Be sure to explain technical terms and their relevance to the patient's condition or the research 
in layman's terms, but without oversimplifying the underlying concepts."""

user_prompt_prepend = """Given the following SHAP values and feature data from an SVR model predicting the drug response to erlotinib, explain the significance of the top features contributing
to the predicted Area Above the Curve (AAC). Ensure the explanation is framed in a formal, scientific tone, as if addressing a clinical research team.

Input data : 
{feature_data}

Prediction : {prediction:.3f}

Frame your explanation to highlight:
The biological relevance of the identified features in the context of drug response.
What are the conclusions that can be drawn from the SHAP values about the drug response prediction.
Don't add any additional part or introduction.
"""


indices = np.concatenate((top_indices, median_indices, bottom_indices))


chats = []
for i in range(len(shap_tables)):
    chat = user_prompt_prepend.format(
        feature_data=shap_tables[i].to_markdown(index=False),
        prediction=y_submission[indices[i]]
    )
    chat = generator.chat_to_dict(chat)
    chat = generator.add_system_prompt(system_prompt, chat)
    chat = generator.apply_chat_template(chat, tokenizer)
    
    chats.append(chat)

In [29]:
print(chats[4])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a highly knowledgeable medical expert with expertise in genomics, bioinformatics, and machine learning. 
You are tasked with explaining the impact of specific genetic features on a prediction model to a patient or researcher. 
Your explanation should be highly formal, precise, and delivered in a professional tone. Be sure to explain technical terms and their relevance to the patient's condition or the research 
in layman's terms, but without oversimplifying the underlying concepts.<|eot_id|><|start_header_id|>user<|end_header_id|>

Given the following SHAP values and feature data from an SVR model predicting the drug response to erlotinib, explain the significance of the top features contributing
to the predicted Area Above the Curve (AAC). Ensure the explanation is framed in a formal, scientific tone, as if addressing a clinical research team.

Input data

In [ ]:
for i in tqdm(range(len(chats))):
    answer = generator.generate({"inputs" : chats[i]}, max_new_tokens=1024)
    #Make sure the directory exists
    os.makedirs("./explanations", exist_ok=True)
    
    with open(f"./explanations/explanation_{i}.txt", "w") as text_file:
        text_file.write(answer[0])



  0%|          | 0/15 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


tensor([358], device='cuda:0')


3395

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

tensor([360], device='cuda:0')


3458

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


tensor([362], device='cuda:0')


2903